In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train_data_task1.csv
/kaggle/input/dev_data_task3.csv
/kaggle/input/train_data_task2.csv
/kaggle/input/dev_data_task1.csv
/kaggle/input/train_data_task3.csv
/kaggle/input/EXIT2024_tweet_test.csv
/kaggle/input/dev_data_task2.csv


In [ ]:
!rm -rf /kaggle/working/output/best-model/task2/hard/*

In [2]:
!wandb offline
!wandb disabled

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.
W&B disabled.


In [3]:
!pip install datasets
!pip install transformers

In [4]:
from  IPython. display import clear_output
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm
import torch
import torch.optim as optim
import torch. nn.functional as F
import re
import warnings
warnings.filterwarnings('ignore')

2024-05-03 09:28:11.110959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 09:28:11.111061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 09:28:11.291240: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
train_df = pd.read_csv('/kaggle/input/train_data_task2.csv')
dev_df = pd.read_csv('/kaggle/input/dev_data_task2.csv')
test_df = pd.read_csv('/kaggle/input/EXIT2024_tweet_test.csv')


In [7]:
device = torch. device('cuda:0'  if  torch.cuda.is_available() else  'cpu')
print(f"computation will run on {device} now")

computation will run on cuda:0 now


# Hard with ALL

In [6]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
    """
    pass the tweet data as a series. do not use apply function
    only preprocesses for replacing @USER and URLS
    """
    # print("i am preprocessing")
    URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
    HANDLE_RE = re.compile(r"@\w+")
    tweets = []
    for t in text:
        t = HANDLE_RE.sub("@USER", t)
        t = URL_RE.sub("HTTPURL", t)
        tweets.append(t)
    return tweets

In [7]:
from sklearn.preprocessing import LabelEncoder

task_encoder = LabelEncoder()

def task_hard_encode(df):
    task_encoder.fit(all_task_hard_labels)
    df['hard_label'] = task_encoder.transform(df['hard_label'])
    return df

def task_hard_decode(df):
    df["hard_label"] = task_encoder.inverse_transform(df["hard_label"])
    return df

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

In [9]:
og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

all_task_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]])
train1_df = task_hard_encode(og_train1)

# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 4]
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

dev1_df = task_hard_encode(og_dev1)
dev1_df = dev1_df[["tweet","hard_label"]].dropna()
dev1_df = dev1_df[dev1_df['hard_label'] != 4]
dev1_df["tweet"] = simple_preprocess(dev1_df["tweet"])

test1_df = og_test1
test1_df = test1_df[["tweet"]]
# test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (5496, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @USER Ignora al otro, es un capullo.El problem...           3
1     @USER Si comicsgate se parece en algo a gamerg...           2
2     @USER Lee sobre Gamergate, y como eso ha cambi...           2
5     @USER Aaah sí. Andrew Dobson. El que se dedicó...           2
7     @USER Esta gringa sigue llorando por el gamerg...           0
...                                                 ...         ...
6913  @USER Ma'am if I say that you look like a whor...           0
6915  idk why y’all bitches think having half your a...           0
6916  This has been a part of an experiment with @US...           1
6917  "Take me already" "Not yet. You gotta be ready...           0
6918    @USER why do you look like a whore? /lh HTTPURL           0

[5496 rows x 2 columns]>
test1 (2076, 1)
<bound method NDFrame.head of                                                   tweet
0     @US

In [10]:
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [11]:
# Load the tokenizer and model
model_name = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Set parameters
MAX_LENGTH = 512

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model/task2/hard",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5386 entries, 0 to 5385
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet       5386 non-null   object
 1   hard_label  5386 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 84.3+ KB


In [ ]:
from datasets import Dataset

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})


In [16]:
from sklearn.metrics import accuracy_score, f1_score

def compute_f1_score(pred):
    # pred is a tuple (predictions, labels)
    predictions, labels = pred
    # Compute the F1 score
    f1 = f1_score(labels, predictions.argmax(axis=1), average='macro')
    return {"f1_score": f1}

In [17]:
# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_f1_score
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss,F1 Score
500,0.970000,0.698068,0.511918
1000,0.710000,0.684344,0.525622


TrainOutput(global_step=1348, training_loss=0.6810504675618972, metrics={'train_runtime': 1240.8371, 'train_samples_per_second': 17.362, 'train_steps_per_second': 1.086, 'total_flos': 5668566366388224.0, 'train_loss': 0.6810504675618972, 'epoch': 4.0})

In [18]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score


# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = val_dataset["label"]

val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels,average = "weighted")

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.7413249211356467
Validation F1 Score: 0.734206252794999


In [19]:
# Get predictions on the test set
test_predictions = trainer.predict(test_dataset)
test_pred_labels = np.argmax(test_predictions.predictions, axis=1)
test_pred_probabilities = softmax(test_predictions.predictions).tolist()

In [20]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

test_probs_list = create_results(test_pred_probabilities, task_encoder.classes_)
test_probs_list[:10]

[{'DIRECT': 1.5572142729070038e-05,
  'JUDGEMENTAL': 7.633352652192116e-06,
  'NO': 0.0011840082006528974,
  'REPORTED': 4.650687969842693e-06},
 {'DIRECT': 6.711523747071624e-05,
  'JUDGEMENTAL': 9.242992746294476e-06,
  'NO': 0.0003228351124562323,
  'REPORTED': 6.197302354848944e-06},
 {'DIRECT': 8.22638139652554e-06,
  'JUDGEMENTAL': 6.4308251239708625e-06,
  'NO': 0.001414161641150713,
  'REPORTED': 4.969675501342863e-06},
 {'DIRECT': 4.229246769682504e-06,
  'JUDGEMENTAL': 2.1560654204222374e-05,
  'NO': 5.067818347015418e-05,
  'REPORTED': 2.0852809029747732e-05},
 {'DIRECT': 9.722945833345875e-06,
  'JUDGEMENTAL': 5.306779257807648e-06,
  'NO': 0.0014551131753250957,
  'REPORTED': 5.2154150580463465e-06},
 {'DIRECT': 1.1625937986536883e-05,
  'JUDGEMENTAL': 6.268676770559978e-06,
  'NO': 0.0013686357997357845,
  'REPORTED': 4.7105486373766325e-06},
 {'DIRECT': 6.01058218308026e-06,
  'JUDGEMENTAL': 3.299143645563163e-05,
  'NO': 7.870394256315194e-06,
  'REPORTED': 3.9478516555

In [21]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    print("output generated as json")
    with open('./output/task2-NICA_hard-2.json', 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels,columns=["hard_label"]), test_probs_list, og_test1[["id"]],og_test1)

decoded labels length: 2076
output generated as json


# Soft with ALL

In [24]:
train_df = pd.read_csv('/kaggle/input/train_data_task2.csv')
dev_df = pd.read_csv('/kaggle/input/dev_data_task2.csv')
test_df = pd.read_csv('/kaggle/input/EXIT2024_tweet_test.csv')

In [22]:
import re

def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
    """
    pass the tweet data as a series. do not use apply function
    only preprocesses for replacing @USER and URLS
    """
    URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
    HANDLE_RE = re.compile(r"@\w+")
    tweets = []
    for t in text:
        t = HANDLE_RE.sub("@USER", t)
        t = URL_RE.sub("HTTPURL", t)
        tweets.append(t)
    return tweets

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are 
    REPORTED, NO, DIRECT, JUDGEMENTAL 
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["REPORTED"], logits_dict["NO"],logits_dict["DIRECT"],logits_dict["JUDGEMENTAL"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"REPORTED": logits_list[0], "NO": logits_list[1],"DIRECT": logits_list[2], "JUDGEMENTAL": logits_list[3]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

In [25]:
og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
train1_df = og_train1[["tweet","soft_label"]].dropna()

og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
dev1_df = og_dev1[["tweet","soft_label"]].dropna()


test1_df = og_test1
test1_df = test1_df[["tweet"]]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (6920, 2)
<bound method NDFrame.head of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.5, 0.16666666666666602, 0.0, 0.333333333333...  
1     [0.0, 0.833333333333333, 0.16666666666666602, ...  
2                                  [0.0, 1.0, 0.0, 0.0]  
3     [0.333

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [27]:
# Set parameters
MAX_LENGTH = 512

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model/task2/soft",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [28]:
import torch.nn as nn
from datasets import Dataset

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})

In [29]:
def custom_loss_fn(logits, soft_labels):
    probs = F.softmax(logits, dim=1)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [30]:
# Define the trainer for each fold
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss
500,19.351500,18.725489
1000,18.620600,18.605980
1500,18.431900,18.580889


TrainOutput(global_step=1692, training_loss=18.670345225232712, metrics={'train_runtime': 1589.1269, 'train_samples_per_second': 17.026, 'train_steps_per_second': 1.065, 'total_flos': 7118860546277376.0, 'train_loss': 18.670345225232712, 'epoch': 4.0})

In [31]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score
# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_logit_labels = val_predictions.predictions
val_pred_probabilities = softmax(val_pred_logit_labels, axis=1)

val_pred_hard_labels = np.argmax(val_pred_probabilities, axis=1)
val_true_labels = val_dataset["label"]
val_true_hard_labels = np.argmax(val_dataset["label"],axis=1)

# Calculate evaluation metrics for each fold
val_accuracy = accuracy_score(val_true_hard_labels, val_pred_hard_labels)
val_f1_score = f1_score(val_true_hard_labels, val_pred_hard_labels,average='weighted')

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.7395309882747069
Validation F1 Score: 0.7297612856318532


In [32]:
# Get predictions on the test set
test_predictions1 = trainer.predict(test_dataset)
test_pred_labels1 = np.argmax(test_predictions1.predictions, axis=1)
test_pred_probabilities1 = softmax(test_predictions1.predictions).tolist()

In [33]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

classes_labels = ["REPORTED", "NO", "DIRECT", "JUDGEMENTAL"]

test_probs_list1 = create_results(test_pred_probabilities1, classes_labels)
test_probs_list1

[{'REPORTED': 1.8971172721649054e-07,
  'NO': 0.0007572341710329056,
  'DIRECT': 1.5727076743132784e-06,
  'JUDGEMENTAL': 2.6623149551596725e-07},
 {'REPORTED': 1.9323182698371966e-07,
  'NO': 0.0005729456897825003,
  'DIRECT': 2.9026348329352913e-06,
  'JUDGEMENTAL': 2.1983933606861683e-07},
 {'REPORTED': 2.6150794951718126e-07,
  'NO': 0.0009282052051275969,
  'DIRECT': 6.443186748583685e-07,
  'JUDGEMENTAL': 3.3146793043670186e-07},
 {'REPORTED': 1.3246993830762221e-06,
  'NO': 0.00044102800893597305,
  'DIRECT': 8.122901817841921e-08,
  'JUDGEMENTAL': 1.3405081062956015e-06},
 {'REPORTED': 2.849249653991137e-07,
  'NO': 0.0008788727573119104,
  'DIRECT': 6.697856633763877e-07,
  'JUDGEMENTAL': 3.2348143008675834e-07},
 {'REPORTED': 2.6243262141179e-07,
  'NO': 0.0008870998281054199,
  'DIRECT': 9.521295396552887e-07,
  'JUDGEMENTAL': 2.87366759721408e-07},
 {'REPORTED': 1.752013668010477e-05,
  'NO': 2.462129486957565e-06,
  'DIRECT': 5.269220082482207e-07,
  'JUDGEMENTAL': 1.27267

In [34]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "./output/task2_NICA-soft-2.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels1,columns=["hard_label"]), test_probs_list1, og_test1[["id"]],og_test1)

decoded labels length: 2076
output generated as json
